# Model - XGRBoost

In [2]:
%%time
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import xgboost as xgb

# -------------------------
# Parameter grid
# -------------------------
parameters = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.001, 0.005, 0.01, 0.05],
    'max_depth': [8, 10, 12, 15],
    'gamma': [0.001, 0.005, 0.01, 0.02],
    'random_state': [42]
}

# -------------------------
# Time-series CV
# -------------------------
tscv = TimeSeriesSplit(n_splits=5)

# -------------------------
# Model
# -------------------------
model = xgb.XGBRegressor(objective='reg:squarederror')

# -------------------------
# GridSearchCV
# -------------------------
clf = GridSearchCV(
    estimator=model,
    param_grid=parameters,
    scoring='neg_mean_squared_error',   # (or neg_mean_absolute_error)
    cv=tscv,
    verbose=1,
    n_jobs=-1
)

# -------------------------
# Fit with eval_set
# -------------------------
eval_set = [(X_train, y_train), (X_valid, y_valid)]

clf.fit(
    X_train,
    y_train,
    eval_set=eval_set,
    verbose=False
)

# -------------------------
# Results
# -------------------------
print(f"Best params: {clf.best_params_}")
print(f"Best CV score (MSE): {clf.best_score_}")

ModuleNotFoundError: No module named 'sklearn'